In [ ]:
import os
import matplotlib.pyplot
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, \
    Dropout, GlobalMaxPooling2D, Activation, Rescaling

physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    "cifar10",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,  # will return tuple (img, label) otherwise dict
    with_info=True,  # able to get info about dataset
)

In [ ]:
image_shape = ds_info.features["image"].shape
print(f'Shape of Images in the Dataset: \t{image_shape}')

num_classes = ds_info.features["label"].num_classes
print(f'Number of Classes in the Dataset: \t{num_classes}')

names_of_classes = ds_info.features["label"].names
print(f'Names of Classes in the Dataset: \t{names_of_classes}\n')

for name in names_of_classes:
    print(f'Label for class \
          "{name}":  \t\t{ds_info.features["label"].str2int(name)}')

In [ ]:
print(f'Total examples in Train Dataset: \
      \t{len(ds_train)}')
print(f'Total examples in Test Dataset: \
      \t{len(ds_test)}')

In [ ]:
def normalize_img(image, label):
    """Normalizes images"""
    return tf.cast(image, tf.float32) / 255.0, label

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 128

In [ ]:
# Setup for train dataset
ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

In [ ]:
# Setup for test Dataset
ds_test = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_train.batch(128)
ds_test = ds_train.prefetch(AUTOTUNE)

In [ ]:
model = keras.Sequential(
    [
        keras.Input((32, 32, 3)),
        layers.Conv2D(32, 3, activation="relu", strides=2),
        layers.Conv2D(64, 3, activation='relu', strides=2),
        layers.Conv2D(128, 3, activation='relu', strides=2),
        layers.Flatten(),
        Dropout(rate=0.5),
        layers.Dense(1024, activation="relu"),
        Dropout(rate=0.2),
        tf.keras.layers.Dense(10, activation="softmax"),
    ]
)

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(0.001),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

In [ ]:
history = model.fit(ds_train, validation_data=ds_test, epochs=15, verbose=2)

In [ ]:
import matplotlib.pyplot as plt

def plot_trend_by_epoch(tr_values, val_values, title):
        epoch_number = range(len(tr_values))
        plt.plot(epoch_number, tr_values, 'r')
        plt.plot(epoch_number, val_values, 'b')
        plt.title(title)
        plt.xlabel('epochs')
        plt.legend(['Training '+title, 'Validation '+title])
        plt.figure()
hist_dict = history.history
tr_accuracy, val_accuracy = hist_dict['accuracy'], \
                                hist_dict['val_accuracy']
plot_trend_by_epoch(tr_accuracy, val_accuracy, "Accuracy")

In [ ]:
tr_loss, val_loss = hist_dict['loss'], hist_dict['val_loss']
plot_trend_by_epoch(tr_loss, val_loss, "Loss")

In [ ]:
import numpy as np

test_labels = []
test_images = []
for image, label in tfds.as_numpy(ds_test.unbatch()):
        test_images.append(image)
        test_labels.append(label)
test_labels = np.array(test_labels)
predictions = model.predict\
                  (ds_test).argmax(axis=1)
incorrect_predictions = np.where(predictions != test_labels)[0]
index = np.random.choice(incorrect_predictions)
plt.imshow(test_images[index])
print(f'True label: {names_of_classes[test_labels[index]]}')
print(f'Predicted label: {names_of_classes[predictions[index]]}')